# <center>JSONiq Update Facility sandbox</center>


The following cell is meant for Google Colab, which does not have the right Java version.

In [ ]:
# We make sure Java 17 is installed.
!apt update
!apt install openjdk-17-jdk
!java -version

In [1]:
!pip install jsoniq


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [38]:
import os
print(os.environ["PATH"])

/Users/ghislain/.pyenv/versions/3.11.9/bin:/opt/homebrew/Cellar/pyenv/2.6.3/libexec:/opt/homebrew/Cellar/pyenv/2.6.3/plugins/python-build/bin:/Users/ghislain/bin/spark-4.0.0-bin-hadoop3/bin:/Users/ghislain/.pyenv/shims:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/usr/local/MacGPG2/bin:/Library/TeX/texbin


We now initialize a RumbleDB session and test that it works.

In [1]:
from jsoniq import RumbleSession

rumble = RumbleSession.builder.withDelta().getOrCreate();
rumble.getRumbleConf().

print(rumble.jsoniq('{ "foo": [ 6*7 ] }').json());

SyntaxError: invalid syntax (2125513810.py, line 4)

For convenience, we also activate the JSONiq magic.

In [2]:
%load_ext jsoniqmagic


We create a new collection

In [9]:
%%jsoniq -pdf
({"foo"||"bar":"bar2"},{"foo":"bar", "bar":[1,2]},{"foo":"asdf"})

,bar,foo,foobar
0,None,None,bar2
1,"[1, 2]",bar,None
2,None,asdf,None


In [13]:
print(rumble.jsoniq('{ "results" : $c.foo[[2]] }',
    c=({"foo":[1,2,3]},{"foo":[4,{"bar":[1,False, None]},6]})).json())

({'results': [2, {'bar': [1, False, None]}]},)


In [14]:
rumble.bind('$c', (1,2,3,4, 5, 6))
print(rumble.jsoniq("""
for $v in $c
let $parity := $v mod 2
group by $parity
return { switch($parity)
         case 0 return "even"
         case 1 return "odd"
         default return "?" : $v
}
""").json())

print(rumble.jsoniq("""
for $v in $c
let $parity := $v mod 2
group by $parity
return { switch($parity)
         case 0 return "gerade"
         case 1 return "ungerade"
         default return "?" : $v
}
""").json())

({'even': [2, 4, 6]}, {'odd': [1, 3, 5]})
({'gerade': [2, 4, 6]}, {'ungerade': [1, 3, 5]})


We insert new data -- remember the snapshot semantics, all updates are computed against the original table, and then applied jointly.

In [15]:
rumble.unbind('$c')

In [20]:
import pandas as pd
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [30,25,35]};
pdf = pd.DataFrame(data);

seq = rumble.jsoniq('$a.Name', a=pdf)
seq.pdf()

***************
Parsing program
***************
***************
Pruning modules
***************
**********************
Resolving dependencies
**********************
*************************************
Populating sequential classifications
*************************************
***************
Expression tree
***************
Unset execution modes: 8
MainModule | unset
  Prolog | unset
    VariableDeclaration (a, external, <anonymous>({ "kind": "object", "baseType": "js:object", "treeDepth": 3, "closed": true, "content": [ { "name": "Name", "type": "xs:string" }, { "name": "Age", "type": "xs:integer" } ] })*)  | unset
  Program | unset
    StatementsAndOptionalExpr | unset | unset | not set
      ObjectLookupExpression | unset | unset | not set
        VariableReferenceExpression ($a)  | unset | unset | not set
        StringLiteralExpression (Name)  | unset | unset | not set


Static context with variables:
Static context with user-defined functions:
Static context with user-defined ty

,__value
0,Alice
1,Bob
2,Charlie


In [19]:

conf = rumble.getRumbleConf()

conf.setPrintIteratorTree(True)

We delete a record

In [11]:
%%jsoniq -u -t
delete table("foo")[$$.foo eq 1500000] from collection

Updates applied successfully.
Response time: 1.3398218154907227 ms


In [12]:
%%jsoniq -df -t
count(table("foo"))

+--------+
|   value|
+--------+
|11000004|
+--------+

Response time: 0.22942113876342773 ms


We edit a record

In [13]:
%%jsoniq -u -t
edit table("foo")[$$.foo eq 1400000] into { "foo" : 22170000, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.
Response time: 1.5862219333648682 ms


In [14]:
%%jsoniq
table("foo")[$$.foo eq 22170000]

{
  "foo": 22170000,
  "bar": [
    "Alice",
    "Bob",
    "Daniel"
  ]
}


In [15]:
%%jsoniq -df -t
table("foo")

+-------+--------------------+-----------+----------+-----------------+--------+---------------+
|    foo|                 bar|    __rowID|__rowOrder|__mutabilityLevel|__pathIn|__tableLocation|
+-------+--------------------+-----------+----------+-----------------+--------+---------------+
|4000000|[Alice, Bob, Char...|17183869187| -99999.95|                0|        |            foo|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|                0|        |            foo|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|                0|        |            foo|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|                0|        |            foo|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|                0|        |            foo|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|                0|        |            foo|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|                0|        |            foo|
|4000007|[Alice, Bob, Char...|

In [16]:
%%jsoniq -t
rename json table("foo")[1].foo as "bar2";

Response time: 3.707296133041382 ms


In [17]:
%%jsoniq -df -t
table("foo")

+-------+--------------------+-----------+----------+-------+-----------------+--------+---------------+
|    foo|                 bar|    __rowID|__rowOrder|   bar2|__mutabilityLevel|__pathIn|__tableLocation|
+-------+--------------------+-----------+----------+-------+-----------------+--------+---------------+
|   NULL|[Alice, Bob, Char...|17183869187| -99999.95|4000000|                0|        |            foo|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|   NULL|                0|        |            foo|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|   NULL|                0|        |            foo|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|   NULL|                0|        |            foo|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|   NULL|                0|        |            foo|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|   NULL|                0|        |            foo|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|  

In [18]:
%%jsoniq -u -t
delete collection table("foo")

Updates applied successfully.
Response time: 0.035366058349609375 ms


In [28]:
%%jsoniq -u -t
create collection delta-file("foobar") with for $i in 1000000 to 3000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }


Updates applied successfully.
Response time: 0.905670166015625 ms


In [29]:
%%jsoniq -pdf -t
delta-file("foobar")

+-------+--------------------+-------+----------+-----------------+--------+--------------------+
|    foo|                 bar|__rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+-------+--------------------+-------+----------+-----------------+--------+--------------------+
|1000000|[Alice, Bob, Char...|      0|       0.0|                0|        |delta.`file:/User...|
|1000001|[Alice, Bob, Char...|      1|       1.0|                0|        |delta.`file:/User...|
|1000002|[Alice, Bob, Char...|      2|       2.0|                0|        |delta.`file:/User...|
|1000003|[Alice, Bob, Char...|      3|       3.0|                0|        |delta.`file:/User...|
|1000004|[Alice, Bob, Char...|      4|       4.0|                0|        |delta.`file:/User...|
|1000005|[Alice, Bob, Char...|      5|       5.0|                0|        |delta.`file:/User...|
|1000006|[Alice, Bob, Char...|      6|       6.0|                0|        |delta.`file:/User...|
|1000007|[Alice, Bob

In [30]:
%%jsoniq -u -t
insert for $i in 4000000 to 6000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  first into collection delta-file("foobar"),
insert for $i in 7000000 to 10000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  last into collection delta-file("foobar"),
insert for $i in 11000000 to 13000000 return { "foo" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  before delta-file("foobar")[position() eq 2] into collection,
insert for $i in 14000000 to 16000000 return { "foobar" : $i, "bar" : [ "Alice", "Bob", "Charlie" ] }
  after delta-file("foobar")[position() eq 2] into collection


Updates applied successfully.
Response time: 12.24434494972229 ms


In [31]:
%%jsoniq -df -t
delta-file("foobar")

+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|    foo|                 bar|    __rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|4000000|[Alice, Bob, Char...|17183869187| -99999.95|                0|        |delta.`file:/User...|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|                0|        |delta.`file:/User...|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|                0|        |delta.`file:/User...|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|                0|        |delta.`file:/User...|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|                0|        |delta.`file:/User...|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|                0|        |delta.`file:/User...|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|                0|        |de

In [32]:
%%jsoniq -u -t
delete delta-file("foobar")[$$.foo eq 15000000] from collection

Updates applied successfully.
Response time: 1.5392799377441406 ms


In [33]:
%%jsoniq -df -t
delta-file("foobar")

+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|    foo|                 bar|    __rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|4000000|[Alice, Bob, Char...|17183869187| -99999.95|                0|        |delta.`file:/User...|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|                0|        |delta.`file:/User...|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|                0|        |delta.`file:/User...|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|                0|        |delta.`file:/User...|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|                0|        |delta.`file:/User...|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|                0|        |delta.`file:/User...|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|                0|        |de

In [35]:
%%jsoniq -u -t
edit delta-file("foobar")[$$.foo eq 1400000] into { "foo" : 17000000, "bar" : [ "Alice", "Bob", "Daniel" ] } in collection

Updates applied successfully.
Response time: 1.647326946258545 ms


In [36]:
%%jsoniq -df -t
delta-file("foobar")

+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|    foo|                 bar|    __rowID|__rowOrder|__mutabilityLevel|__pathIn|     __tableLocation|
+-------+--------------------+-----------+----------+-----------------+--------+--------------------+
|4000000|[Alice, Bob, Char...|17183869187| -99999.95|                0|        |delta.`file:/User...|
|4000001|[Alice, Bob, Char...|17183869188|  -99999.9|                0|        |delta.`file:/User...|
|4000002|[Alice, Bob, Char...|17183869189| -99999.85|                0|        |delta.`file:/User...|
|4000003|[Alice, Bob, Char...|17183869190|  -99999.8|                0|        |delta.`file:/User...|
|4000004|[Alice, Bob, Char...|17183869191| -99999.75|                0|        |delta.`file:/User...|
|4000005|[Alice, Bob, Char...|17183869192|  -99999.7|                0|        |delta.`file:/User...|
|4000006|[Alice, Bob, Char...|17183869193| -99999.65|                0|        |de

In [37]:
%%jsoniq -u -t
delete collection delta-file("foobar")

Updates applied successfully.
Response time: 0.03163290023803711 ms
